In [1]:
import pandas as pd

In [2]:
# Creació de df data a partir del excel
data = pd.read_excel('dades.xlsx', sheet_name = 'Fig-5 Expression values', engine = 'openpyxl')

/home/egarve/miniconda3/envs/tfm/lib/python3.12/site-packages/openpyxl/worksheet/_read_only.py:85: UserWarning: Unknown extension is not supported and will be removed
  for idx, row in parser.parse():


In [3]:
# Eliminació de columna redundant
data = data.drop(columns=["protein name"])

In [4]:
# Eliminació de columnes ambs dades sobre log2 i p value
col_remove = [col for col in data.columns if "log2" in col.lower() or "p value" in col.lower()]
data = data.drop(columns=col_remove)

In [5]:
# Llista de gens del Jupyter Notebook emprat en Neko amb modificacions
genes = [
    "NRG1", "NRG2", "NRG3", "NRG4",
    "EGFR", "ERBB2", "ERBB3", "ERBB4", 
    "PLCG1", "PIK3CA", "PIK3R1", 
    "PTEN", "PDPK1", 
    "PRKCA", "PRKCB", "PRKCG", "PRKCZ",
    "AKT1", "AKT2", "AKT3",
    "TSC2", "TSC1", "RHEB", 
    "MLST8", "MTOR", "RPTOR", "AKT1S1",
    "RICTOR", "MAPKAP1", "PRR5",
    "EIF4EBP1", "DEPTOR",
    "PTK2B", "SRC", "PTPN11", "PRKACA", 
    "SHC1", "GRB2", "SOS1",
    "ARAF", "BRAF", "RAF1",
    "HRAS", "KRAS", "NRAS", 
    "MAP2K1", "MAP2K2",  
    "MAPK3", "MAPK1", 
    "RPS6KA1", "CREB1",
    "ARHGEF1", "ARHGEF6", "ARHGEF11", "ARHGEF12", 
    "CDC42", "CDC42BPA", "CDC42BPB",
    "RAC1", "RAC2", "RAC3", "RHOA", 
    "PAK1", "PAK2", "PAK3", "PAK4", 
    "ROCK1", "ROCK2", "LIMK1", "LIMK2", 
    "CFL1", "CFL2", "STMN1",
    "CDC42EP4", "PPP1R12A", "PPP1R12B"
]

# EGF: en la figura s'indica que no tenen dades, el llevem.
# Complexos: apareixen els components, no els complexos complets, així que els llevem.

In [6]:
# Calculem el nombre de files que hauríem d'obtindre després de filtrar
len(genes)

76

In [7]:
# Filtrem gens de la llista genes
# Afegim passar tot a mayúscules per als gens que no ho estiguen
data = data[data["gene name"].str.upper().isin(genes)]
len(data)

76

In [8]:
# Eliminem la columna de dades que no tinga cap (tot NA)
data = data.dropna(axis='columns', how="all")

# Phospho-Site Ratio (G4/G3) - Inhibition

In [9]:
# Fosforilacions inhibitòries  que es volen filtrar (Fig. 5 Forget et al.)
fosfo_inh = ["S294", "T246", "T70", "S25", "T646"]

In [10]:
# Creem un nou df amb les mateixe columnes
inh = pd.DataFrame(columns=data.columns)

# Busquem les fosforilacions de la llista fosfo_inh en el df data
# Si les troba, copia la fila sencera en el df inh
for index, row in data.iterrows():
    if any(fosfo in str(row.values) for fosfo in fosfo_inh):
        inh = pd.concat([inh, row.to_frame().T], ignore_index=True)

In [11]:
# Eliminem la columna de mRNA i la de proteina
inh = inh.drop(columns=["mRNA ratio (G4 vs. G3)", "protein Ratio (G4 vs. G3)"])

In [12]:
# Creem un nou df per a guardar les dades filtrades
df_inh = []

# Totes les fosforilacions desitjades están al Phospho-Site-1 
# a excepció de la PPP1R12B, que està a la Phospho-Site-2
for index, row in inh.iterrows():
    gene_name = row["gene name"]
    if gene_name == "PPP1R12B":
        # Si es PPP1R12B, agafem phospho-site 2 i la seua ratio
        phospho_site = row["Phospho-Site-2"]
        ratio = row["Ratio (G4 vs. G3) phospho-Site-2"]
        if pd.notna(phospho_site) and pd.notna(ratio):
            df_inh.append([gene_name, phospho_site, ratio])
    else:
        # Per als demés, agafem sols phospho-site 1
        phospho_site = row["Phospho-Site-1"]
        ratio = row["Ratio (G4 vs. G3) phospho-Site-1"]
        if pd.notna(phospho_site) and pd.notna(ratio):
            df_inh.append([gene_name, phospho_site, ratio])

# Guardem dades filtrades en el df inh
inh = pd.DataFrame(df_inh, columns=["gene name", "phospho site", "ratio"])

# Eliminem phospho site
inh = inh.drop(columns=['phospho site'])

# Canviem el nom de "gene name" per "gene"
inh = inh.rename(columns={"gene name": "gene"})

# Resultat
print(inh)

       gene     ratio
0      PTEN  2.706930
1  EIF4EBP1  0.280431
2  PPP1R12B  3.526870
3     STMN1  3.825690
4    AKT1S1  1.875820


# Protein Ratio G4/G3

In [13]:
# Llista de genes_inh
genes_inh = inh["gene"].tolist()
print(genes_inh)

['PTEN', 'EIF4EBP1', 'PPP1R12B', 'STMN1', 'AKT1S1']


In [14]:
# Extracció de columna de protein amb les seues dades al df prot
prot = data[data['protein Ratio (G4 vs. G3)'].notna()]

In [15]:
# Eliminació de genes_inh del df prot
prot = prot[~prot['gene name'].isin(genes_inh)]

# Eliminació de columna mRNA del df prot
prot = prot.drop(columns=['mRNA ratio (G4 vs. G3)'])

# Seleccionem les dues primeres columnes ("gene name" i "protein Ratio (G4 vs. G3)")
prot = prot.iloc[:, :2]

# Canviem el nom de "gene name" per "gene" i de protein Ratio:
prot = prot.rename(columns={"gene name": "gene"})
prot = prot.rename(columns={"protein Ratio (G4 vs. G3)": "ratio"})

print(prot)

         gene     ratio
0        NRG1  6.591660
4        EGFR  2.233910
8        SHC1  0.703137
12       GRB2  1.224990
13       SOS1  0.682682
15       Hras  1.376180
16       Nras  1.855020
17       Kras  3.026130
18       Araf  0.666772
19       Braf  1.592430
20       RAF1  1.585250
21     MAP2K1  1.373440
22     MAP2K2  1.208520
23      MAPK3  2.110630
24      MAPK1  2.748830
28      PLCG1  1.304620
30       RAC1  1.988470
31       RAC2  0.604242
32       RAC3  0.953708
33     PIK3CA  2.891970
36     PIK3R1  1.803950
41      PDPK1  3.074810
42       AKT1  1.565270
43       AKT2  0.952235
44       AKT3  0.983179
45       TSC1  2.481320
46       TSC2  1.861380
52      RPTOR  1.005410
53     RICTOR  0.673088
57        SRC  2.177720
61       mTOR  0.986407
90      PTK2B  9.767110
91     PTPN11  1.832330
98      PRKCA  4.277320
99      PRKCB  6.833150
105      RHOA  1.111550
120     ROCK2  1.475840
121     ROCK1  0.784630
127     CDC42  1.294390
128  CDC42BPB  1.741570
129  CDC42EP4  4

# mRNA Ratio G4/G3

In [16]:
# Llista de genes_prot
genes_prot = prot["gene"].tolist()
print(genes_prot)

['NRG1', 'EGFR', 'SHC1', 'GRB2', 'SOS1', 'Hras', 'Nras', 'Kras', 'Araf', 'Braf', 'RAF1', 'MAP2K1', 'MAP2K2', 'MAPK3', 'MAPK1', 'PLCG1', 'RAC1', 'RAC2', 'RAC3', 'PIK3CA', 'PIK3R1', 'PDPK1', 'AKT1', 'AKT2', 'AKT3', 'TSC1', 'TSC2', 'RPTOR', 'RICTOR', 'SRC', 'mTOR', 'PTK2B', 'PTPN11', 'PRKCA', 'PRKCB', 'RHOA', 'ROCK2', 'ROCK1', 'CDC42', 'CDC42BPB', 'CDC42EP4', 'CDC42BPA', 'PAK1', 'PAK2', 'PAK4', 'PRKACA', 'RPS6KA1', 'ARHGEF1', 'ARHGEF6', 'ARHGEF11', 'ARHGEF12', 'LIMK1', 'CFL1', 'CFL2', 'CREB1', 'PPP1R12A', 'RHEB', 'MLST8']


In [17]:
# Creació de df de mrna i eliminació de genes_inh
mrna = data[~data['gene name'].isin(genes_inh)]

# Elminació de genes_prot
mrna = mrna[~mrna['gene name'].isin(genes_prot)]

# Seleccionem les dues primeres columnes ("gene name" i "mRNA ratio (G4 vs. G3)")
mrna = mrna.iloc[:, :2]

# Canviem el nom de "gene name" per "gene" i de protein Ratio:
mrna = mrna.rename(columns={"gene name": "gene"})
mrna = mrna.rename(columns={"mRNA ratio (G4 vs. G3)": "ratio"})

print(mrna)

        gene      ratio
1       NRG2   5.506050
2       NRG3   0.346638
3       NRG4   4.378950
5      ERBB2   0.717424
6      ERBB3   0.184358
7      ERBB4   6.340570
54    DEPTOR   0.255041
100    PRKCG  19.570000
101    PRKCZ   1.958500
140     PAK3   4.066210
163    LIMK2   0.859694
175     PRR5   3.123560
183  MAPKAP1   0.635370


# Reciprocal Ratio

In [18]:
# Ordre ascendent
inh = inh.sort_values(by=inh.columns[1], ascending=True)
mrna = mrna.sort_values(by=mrna.columns[1], ascending=True)
prot = prot.sort_values(by=prot.columns[1], ascending=True)

In [19]:
# Càlcul de la inversa de la ratio
inh['reciprocal'] = 1 / inh.iloc[:, 1]
mrna['reciprocal'] = 1 / mrna.iloc[:, 1]
prot['reciprocal'] = 1 / prot.iloc[:, 1]

In [20]:
# Extracció d'excel processat
with pd.ExcelWriter('probabilitats.xlsx', engine='openpyxl') as writer:
    inh.to_excel(writer, sheet_name='inh', index=False)
    prot.to_excel(writer, sheet_name='prot', index=False)
    mrna.to_excel(writer, sheet_name='mrna', index=False)